In [1]:
from datetime import datetime
import os, time, re, urllib.parse
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# Selenium options required to create a 'headless' browser
options = Options()
options.add_argument("--blink-settings=imagesEnabled=false")
options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--window-size=1280,800")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.5481.77 Safari/537.37")

driver = webdriver.Chrome(options=options)

In [3]:
def is_plus_code(code):
    # Regex pattern for Plus Codes like "7FG9V3F5+X2" or "2HVW+G8"
    pattern = r'^[2-9A-HJ-NP-Z]{4,7}\+[2-9A-HJ-NP-Z]{2,}$'
    return re.match(pattern, code) is not None

def get_maps_url(origin, destination):
    origin = urllib.parse.quote(origin)
    destination = urllib.parse.quote(destination)
    url = f"https://www.google.com/maps/dir/{origin}/{destination}/data=!3e0"
    return url

def avg_speed(duration, distance):
    distance = float(distance.split(' ')[0])
    d = duration.split(' ')
    duration = int(d[0])*60 if 'hr' in d else 0
    duration += int(d[-2]) if 'min' in d else 0
    return distance/duration*60

def get_traffic_report(origin, destination, mode='car'):
    modes = {'bike': "\ue9f9", 'car': "\ue531", 'transit': "\ue535"}
    if mode in modes.keys():
        mode = modes[mode]
    else:
        mode = modes['car']

    maps_url = get_maps_url(origin, destination)
    print(maps_url)
    driver.get(maps_url)

    routes = driver.find_elements(By.CSS_SELECTOR, "div[data-trip-index]")
    time_taken = routes[0].text.split("\n")[1]
    for route in routes:
        if mode in route.text:
            time_taken = route.text.split("\n")[1]
            distance = route.text.split("\n")[2]
            break

    return time_taken, distance

In [4]:
routes = [
    {"route_code": "2HVW+G8|XJXR+WG",
     "origin": "Bethel AG Church, Hebbal Kempapura, Bengaluru",
     "destination": "SMVT Railway Station, Bengaluru"},

    {"route_code": "XHCW+W2|3H3V+J5",
     "origin": "JW Marriott Hotel, Bengaluru",
     "destination": "Hyatt Centric Hebbal, Bengaluru"},

    {"route_code": "WGH7+QP|XHJ7+MG",
     "origin": "Cult RR Nagar Fitness Center, Bengaluru",
     "destination": "Lulu Mall, Bengaluru"},

    {"route_code": "XMXH+X3J|WMGQ+34",
     "origin": "K R Pura Metro Station, Bengaluru",
     "destination": "RMZ Ecoworld, Bellandur, Bengaluru"},

    {"route_code": "WH5F+26|WJ8X+F5W",
     "origin": "Jaya Prakash Nagar Metro Station, Bengaluru",
     "destination": "Hemavathi Park, HSR Layout, Bengaluru"},

    {"route_code": "XPC7+72|WJP4+FF",
     "origin": "The Rameshwaram Cafe @ Brookfield, Bengaluru",
     "destination": "Gawky Goose, Wind Tunnel Rd, Bengaluru"},

    {"route_code": "XJPW+92|XM33+J3",
     "origin": "Swami Vivekananda Road Metro Station, Bengaluru",
     "destination": "Christ University, Hosur Main Road, Bengaluru"},

    {"route_code": "XJG4+7J|5PX4+HQ",
     "origin": "MG Road Metro Station, Bengaluru",
     "destination": "Kempegowda International Airport Bengaluru"}
]

In [5]:
today = datetime.now()
df = pd.DataFrame()

for route in routes:
    origin = route["origin"]
    destination = route["destination"]
    o_code, d_code = route["route_code"].split("|")
    travel_time, travel_distance = get_traffic_report(origin, destination)

    new_row = {
        "date": today.date(),
        "time": today.time().strftime("%H:%M"),
        "route_code": route["route_code"],
        "from": origin,
        "to": destination,
        "duration": travel_time,
        "distance": travel_distance,
        "avg_speed": avg_speed(travel_time, travel_distance),
    }

    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

# driver.quit()
# out_file = "blr_traffic_times.csv"
# df.to_csv(out_file, mode="a", header=header, index=False)

https://www.google.com/maps/dir/Bethel%20AG%20Church%2C%20Hebbal%20Kempapura%2C%20Bengaluru/SMVT%20Railway%20Station%2C%20Bengaluru/data=!3e0
https://www.google.com/maps/dir/JW%20Marriott%20Hotel%2C%20Bengaluru/Hyatt%20Centric%20Hebbal%2C%20Bengaluru/data=!3e0
https://www.google.com/maps/dir/Cult%20RR%20Nagar%20Fitness%20Center%2C%20Bengaluru/Lulu%20Mall%2C%20Bengaluru/data=!3e0
https://www.google.com/maps/dir/K%20R%20Pura%20Metro%20Station%2C%20Bengaluru/RMZ%20Ecoworld%2C%20Bellandur%2C%20Bengaluru/data=!3e0
https://www.google.com/maps/dir/Jaya%20Prakash%20Nagar%20Metro%20Station%2C%20Bengaluru/Hemavathi%20Park%2C%20HSR%20Layout%2C%20Bengaluru/data=!3e0
https://www.google.com/maps/dir/The%20Rameshwaram%20Cafe%20%40%20Brookfield%2C%20Bengaluru/Gawky%20Goose%2C%20Wind%20Tunnel%20Rd%2C%20Bengaluru/data=!3e0
https://www.google.com/maps/dir/Swami%20Vivekananda%20Road%20Metro%20Station%2C%20Bengaluru/Christ%20University%2C%20Hosur%20Main%20Road%2C%20Bengaluru/data=!3e0
https://www.google.co

In [6]:
display(df)

,date,time,route_code,from,to,duration,distance,avg_speed
0,2025-07-27,03:22,2HVW+G8|XJXR+WG,"Bethel AG Church, Hebbal Kempapura, Bengaluru","SMVT Railway Station, Bengaluru",18 min,9.9 km,33.000000
1,2025-07-27,03:22,XHCW+W2|3H3V+J5,"JW Marriott Hotel, Bengaluru","Hyatt Centric Hebbal, Bengaluru",15 min,10.6 km,42.400000
2,2025-07-27,03:22,WGH7+QP|XHJ7+MG,"Cult RR Nagar Fitness Center, Bengaluru","Lulu Mall, Bengaluru",19 min,10.3 km,32.526316
3,2025-07-27,03:22,XMXH+X3J|WMGQ+34,"K R Pura Metro Station, Bengaluru","RMZ Ecoworld, Bellandur, Bengaluru",20 min,11.1 km,33.300000
4,2025-07-27,03:22,WH5F+26|WJ8X+F5W,"Jaya Prakash Nagar Metro Station, Bengaluru","Hemavathi Park, HSR Layout, Bengaluru",16 min,10.2 km,38.250000
5,2025-07-27,03:22,XPC7+72|XM33+J3,"The Rameshwaram Cafe @ Brookfield, Bengaluru","Gawky Goose, Wind Tunnel Rd, Bengaluru",15 min,9.4 km,37.600000
6,2025-07-27,03:22,XJG4+7J|5PX4+HQ,"MG Road Metro Station, Bengaluru",Kempegowda International Airport Bengaluru,36 min,33.9 km,56.500000


In [12]:
display(df)

,date,time,route_code,from,to,duration,distance,avg_speed
0,2025-07-27,21:01,2HVW+G8|XJXR+WG,"Bethel AG Church, Hebbal Kempapura, Bengaluru","SMVT Railway Station, Bengaluru",30 min,9.7 km,19.400000
1,2025-07-27,21:01,XHCW+W2|3H3V+J5,"JW Marriott Hotel, Bengaluru","Hyatt Centric Hebbal, Bengaluru",22 min,10.3 km,28.090909
2,2025-07-27,21:01,WGH7+QP|XHJ7+MG,"Cult RR Nagar Fitness Center, Bengaluru","Lulu Mall, Bengaluru",27 min,9.3 km,20.666667
3,2025-07-27,21:01,XMXH+X3J|WMGQ+34,"K R Pura Metro Station, Bengaluru","RMZ Ecoworld, Bellandur, Bengaluru",22 min,11.1 km,30.272727
4,2025-07-27,21:01,WH5F+26|WJ8X+F5W,"Jaya Prakash Nagar Metro Station, Bengaluru","Hemavathi Park, HSR Layout, Bengaluru",24 min,10.7 km,26.750000
5,2025-07-27,21:01,XPC7+72|WJP4+FF,"The Rameshwaram Cafe @ Brookfield, Bengaluru","Gawky Goose, Wind Tunnel Rd, Bengaluru",26 min,9.4 km,21.692308
6,2025-07-27,21:01,XJPW+92|XM33+J3,"Swami Vivekananda Road Metro Station, Bengaluru","Christ University, Hosur Main Road, Bengaluru",34 min,9.7 km,17.117647
7,2025-07-27,21:01,XJG4+7J|5PX4+HQ,"MG Road Metro Station, Bengaluru",Kempegowda International Airport Bengaluru,48 min,34.2 km,42.750000


In [6]:
display(df)

,date,time,route_code,from,to,duration,distance,avg_speed
0,2025-07-28,08:22,2HVW+G8|XJXR+WG,"Bethel AG Church, Hebbal Kempapura, Bengaluru","SMVT Railway Station, Bengaluru",42 min,9.9 km,14.142857
1,2025-07-28,08:22,XHCW+W2|3H3V+J5,"JW Marriott Hotel, Bengaluru","Hyatt Centric Hebbal, Bengaluru",22 min,10.4 km,28.363636
2,2025-07-28,08:22,WGH7+QP|XHJ7+MG,"Cult RR Nagar Fitness Center, Bengaluru","Lulu Mall, Bengaluru",29 min,9.3 km,19.241379
3,2025-07-28,08:22,XMXH+X3J|WMGQ+34,"K R Pura Metro Station, Bengaluru","RMZ Ecoworld, Bellandur, Bengaluru",41 min,11.1 km,16.243902
4,2025-07-28,08:22,WH5F+26|WJ8X+F5W,"Jaya Prakash Nagar Metro Station, Bengaluru","Hemavathi Park, HSR Layout, Bengaluru",26 min,10.4 km,24.000000
5,2025-07-28,08:22,XPC7+72|WJP4+FF,"The Rameshwaram Cafe @ Brookfield, Bengaluru","Gawky Goose, Wind Tunnel Rd, Bengaluru",39 min,9.4 km,14.461538
6,2025-07-28,08:22,XJPW+92|XM33+J3,"Swami Vivekananda Road Metro Station, Bengaluru","Christ University, Hosur Main Road, Bengaluru",37 min,10.7 km,17.351351
7,2025-07-28,08:22,XJG4+7J|5PX4+HQ,"MG Road Metro Station, Bengaluru",Kempegowda International Airport Bengaluru,46 min,33.9 km,44.217391


In [12]:
display(df)

,date,time,route_code,from,to,duration,distance,avg_speed
0,2025-07-28,19:18,2HVW+G8|XJXR+WG,"Bethel AG Church, Hebbal Kempapura, Bengaluru","SMVT Railway Station, Bengaluru",39 min,9.9 km,15.230769
1,2025-07-28,19:18,XHCW+W2|3H3V+J5,"JW Marriott Hotel, Bengaluru","Hyatt Centric Hebbal, Bengaluru",40 min,11.7 km,17.550000
2,2025-07-28,19:18,WGH7+QP|XHJ7+MG,"Cult RR Nagar Fitness Center, Bengaluru","Lulu Mall, Bengaluru",35 min,9.3 km,15.942857
3,2025-07-28,19:18,XMXH+X3J|WMGQ+34,"K R Pura Metro Station, Bengaluru","RMZ Ecoworld, Bellandur, Bengaluru",25 min,11.1 km,26.640000
4,2025-07-28,19:18,WH5F+26|WJ8X+F5W,"Jaya Prakash Nagar Metro Station, Bengaluru","Hemavathi Park, HSR Layout, Bengaluru",28 min,10.2 km,21.857143
5,2025-07-28,19:18,XPC7+72|WJP4+FF,"The Rameshwaram Cafe @ Brookfield, Bengaluru","Gawky Goose, Wind Tunnel Rd, Bengaluru",34 min,9.4 km,16.588235
6,2025-07-28,19:18,XJPW+92|XM33+J3,"Swami Vivekananda Road Metro Station, Bengaluru","Christ University, Hosur Main Road, Bengaluru",47 min,10.6 km,13.531915
7,2025-07-28,19:18,XJG4+7J|5PX4+HQ,"MG Road Metro Station, Bengaluru",Kempegowda International Airport Bengaluru,1 hr 8 min,34.4 km,30.352941


In [6]:
display(df)

,date,time,route_code,from,to,duration,distance,avg_speed
0,2025-07-29,19:34,2HVW+G8|XJXR+WG,"Bethel AG Church, Hebbal Kempapura, Bengaluru","SMVT Railway Station, Bengaluru",36 min,9.9 km,16.500000
1,2025-07-29,19:34,XHCW+W2|3H3V+J5,"JW Marriott Hotel, Bengaluru","Hyatt Centric Hebbal, Bengaluru",41 min,10.9 km,15.951220
2,2025-07-29,19:34,WGH7+QP|XHJ7+MG,"Cult RR Nagar Fitness Center, Bengaluru","Lulu Mall, Bengaluru",34 min,9.3 km,16.411765
3,2025-07-29,19:34,XMXH+X3J|WMGQ+34,"K R Pura Metro Station, Bengaluru","RMZ Ecoworld, Bellandur, Bengaluru",28 min,11.1 km,23.785714
4,2025-07-29,19:34,WH5F+26|WJ8X+F5W,"Jaya Prakash Nagar Metro Station, Bengaluru","Hemavathi Park, HSR Layout, Bengaluru",26 min,10.7 km,24.692308
5,2025-07-29,19:34,XPC7+72|WJP4+FF,"The Rameshwaram Cafe @ Brookfield, Bengaluru","Gawky Goose, Wind Tunnel Rd, Bengaluru",37 min,9.4 km,15.243243
6,2025-07-29,19:34,XJPW+92|XM33+J3,"Swami Vivekananda Road Metro Station, Bengaluru","Christ University, Hosur Main Road, Bengaluru",43 min,9.7 km,13.534884
7,2025-07-29,19:34,XJG4+7J|5PX4+HQ,"MG Road Metro Station, Bengaluru",Kempegowda International Airport Bengaluru,1 hr 5 min,34.6 km,31.938462
